In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from serpapi import GoogleSearch

# Load extracted course data
def load_extracted_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

data = load_extracted_data("extracted_course_data.json")
course_title = data.get("course_title", "")
weekly_topics = data.get("weekly_topics", [])
objectives = data.get("objectives", [])
study_materials_listed = data.get("study_materials", [])

SERPAPI_KEY = os.getenv("SERPAPI_KEY")

def google_search(query, num_results=5):
    params = {
        "q": query,
        "num": num_results,
        "api_key": SERPAPI_KEY,
        "engine": "google"
    }
    search = GoogleSearch(params)
    results = search.get()
    links = [result['link'] for result in results.get("organic_results", [])]
    return links

# Get study materials from Google
def recommend_study_materials():
    queries = [course_title] + weekly_topics[:3] + objectives[:3]  # Limit for relevance
    recommendations = {}
    
    for query in queries:
        recommendations[query] = google_search(f"{query} study materials")
    
    return recommendations

# Load past papers and analyze frequent topics
def analyze_past_papers(past_papers):
    topic_counter = Counter()
    for paper in past_papers:
        for topic in weekly_topics:
            if topic.lower() in paper.lower():
                topic_counter[topic] += 1
    
    return topic_counter

# Visualization: Top Exam Topics
def plot_frequent_topics(topic_counter):
    topics, counts = zip(*topic_counter.most_common(10))
    plt.figure(figsize=(10, 5))
    plt.barh(topics, counts, color='skyblue')
    plt.xlabel("Frequency in Past Papers")
    plt.title("Most Frequently Asked Topics in Exams")
    plt.gca().invert_yaxis()
    plt.show()

# Sample past paper data (to be replaced with real input)
past_papers = [
    "Explain supervised learning and its advantages",
    "Discuss reinforcement learning applications",
    "Define deep learning and neural networks",
    "Explain supervised learning with examples",
    "What are convolutional neural networks?"
]

topic_counter = analyze_past_papers(past_papers)
plot_frequent_topics(topic_counter)

# Generate study material recommendations
recommended_study_materials = recommend_study_materials()
print(json.dumps(recommended_study_materials, indent=4))

# Save recommendations to file
with open("recommended_study_materials.json", "w", encoding="utf-8") as f:
    json.dump(recommended_study_materials, f, indent=4)
